# NLP Tasks (Part 1)

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
| 18/12/2025   | Martin | Create  | Notebook created for various NLP tasks using HF | 

# Content

* [Introduction](#introduction)
* [1. Translation](#1-translation)

# Introduction

Tackle common NLP problems using LLMs built using the HF package:

1. Translation
2. Summarisation

# 1. Translation

- Seq-2-Seq task
- Finetune existing language model (mT5, mBART, Marian - here)

<u>Components</u>

- English to French translation model
- KDE4 dataset: Localised files for KDE (Apps for Linux desktops)

In [ ]:
from datasets import load_dataset
from transformers import pipeline

SEED = 20

In [ ]:
raw_datasets = load_dataset("kde4", lang1="en", lang2="fr")
raw_datasets

In [ ]:
split_datasets = raw_datasets['train'].train_test_split(train_size=0.9, seed=SEED)
split_datasets['validation'] = split_datasets.pop('test')
split_datasets

Dataset contains 2 statements one for each language. The KDE4 dataset translates many of the technical terms to the corresponding language, but the pretrained model does not do this

In [ ]:
# KDE4 dataset
split_datasets['train'][1]['translation']

In [ ]:
# Pretrained model
model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
translator = pipeline("translation", model=model_checkpoint)
translator("Default to expanded threads")

In [2]:
%watermark

Last updated: 2025-06-18T19:03:45.452311+08:00

Python implementation: CPython
Python version       : 3.11.9
IPython version      : 8.31.0

Compiler    : MSC v.1938 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 183 Stepping 1, GenuineIntel
CPU cores   : 20
Architecture: 64bit

